In [1]:
import uuid
import requests
import pandas as pd
from arango import ArangoClient

In [2]:
raw_df = pd.read_csv('../data/raw.csv')
raw_df.head()

,country_of_destination,country_of_origin,hs6_code,receiver_address,receiver_id,receiver_name,sender_address,sender_id,sender_name,tier,transaction_date
0,CA,US,321410,"95 SUNRAY STREET, WHITBY, ON, L1N9C9, CANADA T...",4bd999c0cb4346ce047f99a692ec3191,Stonhard Ltd,"60 MARKET SAUQRE, BELIZE CITY, BELIZE",52c8642c34649eafe2d044eee3d884e1,Global Fire Protection,t1->t0,11/30/18
1,NaN,NaN,320890,China,8b1f537a8eeca839cff8a99e08b35690,CARBOLINE DALIAN PAINT COMPANY LTD,"Plot O 356 &357, Sidco Indl Estate Ambattur Ch...",831a31a1466f0ace3eb20b52d4575f92,Carboline (India) Private Limited,t1->t0,5/2/18
2,US,IN,390940,United States,1670c05b29762f5d4ab8980d267bd482,Rust-Oleum Llc,"204, Monarch Chambers, Marol Maroshi Road, Mum...",f580bc1756d06768c94634b0332e1871,Paladin Paints & Chemicals Private Limited,t1->t0,5/2/17
3,US,CN,731100,OF RPM WOOD FINISHES GROUP 3190 HICKORY BLVD H...,210f645e15d75833b25090f2ae509eda,Mohawk Finishing Products Inc,"YUNG-SHI BLVD, SHI-WAN TOWN, PO-LO COUNTY HUI-...",149081be00548b006dc38a88264eae32,Alpha Pacific Group Pte. Ltd.,t1->t0,5/8/16
4,CH,IN,250590,Switzerland,7150428f8927c86ac11c64c1bb9b0285,Vandex International AG,FLOWCRETE INDIA PRIVATE LIMITEDNEW NO.36 OLD N...,f4864ac3d5d716cc586d60afe8a403ef,TREMCO CPG INDIA PRIVATE LIMITED,t1->t0,5/26/17


In [3]:
print(raw_df.country_of_origin.isnull().sum())
print(raw_df.country_of_origin.isnull().sum())

31409
31409


In [4]:
address_df = pd.read_csv('./data/address.csv')

In [5]:
merged_send = pd.merge(raw_df, address_df, left_on=['sender_id', 'sender_address'], right_on=['organization_id', 'address'], how="left", validate="m:1")
merged_receive = pd.merge(raw_df, address_df, left_on=['receiver_id', 'receiver_address'], right_on=['organization_id', 'address'], how="left", validate="m:1")

In [6]:
raw_df['country_of_origin'] = merged_send['country']
raw_df['country_of_destination'] = merged_receive['country']

In [7]:
print(raw_df.country_of_origin.isna().sum())
print(raw_df.country_of_destination.isna().sum())

0
0


In [8]:
raw_df.head(5)

,country_of_destination,country_of_origin,hs6_code,receiver_address,receiver_id,receiver_name,sender_address,sender_id,sender_name,tier,transaction_date
0,CA,US,321410,"95 SUNRAY STREET, WHITBY, ON, L1N9C9, CANADA T...",4bd999c0cb4346ce047f99a692ec3191,Stonhard Ltd,"60 MARKET SAUQRE, BELIZE CITY, BELIZE",52c8642c34649eafe2d044eee3d884e1,Global Fire Protection,t1->t0,11/30/18
1,CN,Other,320890,China,8b1f537a8eeca839cff8a99e08b35690,CARBOLINE DALIAN PAINT COMPANY LTD,"Plot O 356 &357, Sidco Indl Estate Ambattur Ch...",831a31a1466f0ace3eb20b52d4575f92,Carboline (India) Private Limited,t1->t0,5/2/18
2,US,IN,390940,United States,1670c05b29762f5d4ab8980d267bd482,Rust-Oleum Llc,"204, Monarch Chambers, Marol Maroshi Road, Mum...",f580bc1756d06768c94634b0332e1871,Paladin Paints & Chemicals Private Limited,t1->t0,5/2/17
3,US,CN,731100,OF RPM WOOD FINISHES GROUP 3190 HICKORY BLVD H...,210f645e15d75833b25090f2ae509eda,Mohawk Finishing Products Inc,"YUNG-SHI BLVD, SHI-WAN TOWN, PO-LO COUNTY HUI-...",149081be00548b006dc38a88264eae32,Alpha Pacific Group Pte. Ltd.,t1->t0,5/8/16
4,CH,IN,250590,Switzerland,7150428f8927c86ac11c64c1bb9b0285,Vandex International AG,FLOWCRETE INDIA PRIVATE LIMITEDNEW NO.36 OLD N...,f4864ac3d5d716cc586d60afe8a403ef,TREMCO CPG INDIA PRIVATE LIMITED,t1->t0,5/26/17


# Sites

In [9]:
client = ArangoClient(hosts="https://2ae4f052d710.arangodb.cloud:8529")

db = client.db('machine_learning', username='lab_test', password='z-rRdN-Enf4qQwybGiVdbG')

In [10]:
sites = db.collection('sites')

In [11]:
cursor = sites.all()

In [12]:
sites = []
for site in cursor:
    sites.append(site)

In [13]:
sites_df = pd.DataFrame(sites)

In [14]:
sites_df['organization_id'] = sites_df['organization_id'].str.split('organizations/').str[1]
sites_df['country_id'] = sites_df['country_id'].str.split('countries/').str[1]

In [15]:
sender_site = pd.merge(raw_df, sites_df, left_on=['sender_id', 'country_of_origin'], right_on=['organization_id', 'country_id'], validate="m:1")

In [16]:
receiver_site = pd.merge(raw_df, sites_df, left_on=['receiver_id', 'country_of_destination'], right_on=['organization_id', 'country_id'], validate="m:1")

In [17]:
raw_df['sender_site_id'] = sender_site['_key']
raw_df['receiver_site_id'] = receiver_site['_key']

In [18]:
print(raw_df.sender_site_id.isnull().sum())
print(raw_df.receiver_site_id.isnull().sum())

0
0


In [19]:
raw_df

,country_of_destination,country_of_origin,hs6_code,receiver_address,receiver_id,receiver_name,sender_address,sender_id,sender_name,tier,transaction_date,sender_site_id,receiver_site_id
0,CA,US,321410,"95 SUNRAY STREET, WHITBY, ON, L1N9C9, CANADA T...",4bd999c0cb4346ce047f99a692ec3191,Stonhard Ltd,"60 MARKET SAUQRE, BELIZE CITY, BELIZE",52c8642c34649eafe2d044eee3d884e1,Global Fire Protection,t1->t0,11/30/18,18857ddacc1f473bb65d0a42030601a6,1568051a614a4133b0a36dde7ab1139b
1,CN,Other,320890,China,8b1f537a8eeca839cff8a99e08b35690,CARBOLINE DALIAN PAINT COMPANY LTD,"Plot O 356 &357, Sidco Indl Estate Ambattur Ch...",831a31a1466f0ace3eb20b52d4575f92,Carboline (India) Private Limited,t1->t0,5/2/18,8659f74cc67d4c53b80693eaae86c6f1,4b03dcdf9f89410c8179b5748ac9bb5b
2,US,IN,390940,United States,1670c05b29762f5d4ab8980d267bd482,Rust-Oleum Llc,"204, Monarch Chambers, Marol Maroshi Road, Mum...",f580bc1756d06768c94634b0332e1871,Paladin Paints & Chemicals Private Limited,t1->t0,5/2/17,14c27d3af19a40259c5fbea6c774fd00,9af2b9518dfd4689b1261b84742ee251
3,US,CN,731100,OF RPM WOOD FINISHES GROUP 3190 HICKORY BLVD H...,210f645e15d75833b25090f2ae509eda,Mohawk Finishing Products Inc,"YUNG-SHI BLVD, SHI-WAN TOWN, PO-LO COUNTY HUI-...",149081be00548b006dc38a88264eae32,Alpha Pacific Group Pte. Ltd.,t1->t0,5/8/16,540ee69808d3411a8bed3bd349d3f6a4,d77436c4d2c347e2a857b1c2c52f1d91
4,CH,IN,250590,Switzerland,7150428f8927c86ac11c64c1bb9b0285,Vandex International AG,FLOWCRETE INDIA PRIVATE LIMITEDNEW NO.36 OLD N...,f4864ac3d5d716cc586d60afe8a403ef,TREMCO CPG INDIA PRIVATE LIMITED,t1->t0,5/26/17,52b076a7d8c74afaabba4f40af58bf0e,05c982cf09864129913f53372d5eb3fe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99879,KZ,IT,732690,"Satpaeva ul d. 3, Atyrau, Atyrau, 060011, Kaza...",66ebf2a3aa71c90b4df3fc93590e22db,Tengizchevroil Llp,Italy,25fe0d54401de2e86f6ebb4828648e0e,Atb Riva Calzoni S.P.A.,t2->t1,5/10/19,da5595f105264e9f9e4e520073600a83,3b34b76e83354031ae39ae3c5336c510
99880,KZ,IT,732690,"Satpaeva ul d. 3, Atyrau, Atyrau, 060011, Kaza...",66ebf2a3aa71c90b4df3fc93590e22db,Tengizchevroil Llp,Italy,25fe0d54401de2e86f6ebb4828648e0e,Atb Riva Calzoni S.P.A.,t2->t1,5/10/19,da5595f105264e9f9e4e520073600a83,3b34b76e83354031ae39ae3c5336c510
99881,KZ,IT,730791,"Satpaeva ul d. 3, Atyrau, Atyrau, 060011, Kaza...",66ebf2a3aa71c90b4df3fc93590e22db,Tengizchevroil Llp,Italy,25fe0d54401de2e86f6ebb4828648e0e,Atb Riva Calzoni S.P.A.,t2->t1,5/10/19,da5595f105264e9f9e4e520073600a83,3b34b76e83354031ae39ae3c5336c510
99882,KZ,IT,730791,"Satpaeva ul d. 3, Atyrau, Atyrau, 060011, Kaza...",66ebf2a3aa71c90b4df3fc93590e22db,Tengizchevroil Llp,Italy,25fe0d54401de2e86f6ebb4828648e0e,Atb Riva Calzoni S.P.A.,t2->t1,5/10/19,da5595f105264e9f9e4e520073600a83,3b34b76e83354031ae39ae3c5336c510


In [23]:
raw_df.to_csv('./data/processed.csv', index=False)

# Transactions Collection

In [34]:
raw_df.rename(columns={'transaction_date':'date'}, inplace=True)
raw_df['_from'] = 'sites/' + raw_df['sender_site_id']
raw_df['_to'] = 'sites/' + raw_df['receiver_site_id']
raw_df['_key'] = [uuid.uuid4().hex for _ in range(len(raw_df))]

In [35]:
client = ArangoClient(hosts="https://2ae4f052d710.arangodb.cloud:8529")

db = client.db('machine_learning', username='lab_test', password='z-rRdN-Enf4qQwybGiVdbG')

In [41]:
db.delete_collection('transactions')
if not db.has_collection('transactions'):
    transactions = db.create_collection('transactions', edge=True)
else:
    transactions = db.collection('transactions')

In [42]:
transactions_list = raw_df[['_key', '_from', '_to', 'date', 'hs6_code', 'tier']].to_dict(orient='records')

In [43]:
transactions.insert_many(transactions_list, overwrite_mode="replace", silent=True)

[{'_id': 'transactions/cc90c434a2c8421bb133569d4641dd14',
  '_key': 'cc90c434a2c8421bb133569d4641dd14',
  '_rev': '_igzIMpS---'},
 {'_id': 'transactions/ac1ff0082e0f4dafa111826516ff2f33',
  '_key': 'ac1ff0082e0f4dafa111826516ff2f33',
  '_rev': '_igzIMpS--_'},
 {'_id': 'transactions/e943dbf4981849cf8113fcb20003eea6',
  '_key': 'e943dbf4981849cf8113fcb20003eea6',
  '_rev': '_igzIMpS--A'},
 {'_id': 'transactions/b394774bc9434cdca64e157cc9e29875',
  '_key': 'b394774bc9434cdca64e157cc9e29875',
  '_rev': '_igzIMpS--B'},
 {'_id': 'transactions/8891b03f57ba4d8ca77ba62ea21cd7b2',
  '_key': '8891b03f57ba4d8ca77ba62ea21cd7b2',
  '_rev': '_igzIMpS--C'},
 {'_id': 'transactions/cf231da136bb4272820eee23ee2b6fc5',
  '_key': 'cf231da136bb4272820eee23ee2b6fc5',
  '_rev': '_igzIMpS--D'},
 {'_id': 'transactions/0cf45d8037314ec284d7211e5ce15beb',
  '_key': '0cf45d8037314ec284d7211e5ce15beb',
  '_rev': '_igzIMpS--E'},
 {'_id': 'transactions/f77c802bdbf641de9fd39df9e7194b1a',
  '_key': 'f77c802bdbf641de9fd39

In [44]:
assert transactions.count() == raw_df.shape[0]